In [5]:
import numpy as np
import pickle
import cv2 as cv
import os
import tensorflow
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import utils
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D,Dropout,Flatten,Dense,MaxPool2D
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from keras.applications.vgg16 import VGG16, preprocess_input

In [6]:
Categories = ["Anemia" , "Non Anemia"]
data_dir = 'C:\\Users\\HP\\Desktop\\Model deployment\\final dataset'
data = []                                                        
for category in Categories:
  path = os.path.join(data_dir,category)
  class_num = Categories.index(category)
  for img in os.listdir(path):
    img_arr = cv.imread(os.path.join(path,img))
    re = cv.resize(img_arr,(100,100))
    data.append([re,class_num])

In [7]:
import cv2
import numpy as np

# Assuming 'data' is a list of tuples containing (image, label) pairs
np.random.shuffle(data)

x_data = []  # Initialize list to store converted images
y_data = []  # Initialize list to store labels

# Iterate through the data list and append the converted images to x_data and labels to y_data
for features, labels in data:
    # Convert RGB image to LAB color space
    lab_image = cv2.cvtColor(features, cv2.COLOR_RGB2LAB)

    # Append the LAB image to x_data
    x_data.append(lab_image)

    # Append the label to y_data
    y_data.append(labels)

# Convert the lists to NumPy arrays
x_data = np.array(x_data)
y_data = np.array(y_data)

# Print the shapes of x_data and y_data
print(f"Shape of x_data: {x_data.shape}")
print(f"Shape of y_data: {y_data.shape}")


Shape of x_data: (3604, 100, 100, 3)
Shape of y_data: (3604,)


In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import cv2
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the CNN model
input_shape = (100, 100, 3)  # Assuming the user's input image has shape (100, 100, 3)
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), input_shape=input_shape, activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(2, activation='sigmoid'))
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


features = cnn_model.predict(x_data)
features_flatten = features.reshape(features.shape[0], -1)
# Save the model
cnn_model=cnn_model.save('cnn_model.h5')

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y_data)

# Save the label encoder
with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)



#random forest
X_train, X_test, y_train, y_test = train_test_split(features_flatten, encoded_labels, test_size=0.2, random_state=42)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report



# Initialize Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# Perform 10-fold cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in skf.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[test_index]

    # Fit the model on the training fold
    rf_model.fit(X_train_fold, y_train_fold)

    # Predict on the validation fold
    y_pred_val = rf_model.predict(X_val_fold)

    # Evaluate the model on the validation fold
    accuracy_val = accuracy_score(y_val_fold, y_pred_val)
    print(f'Validation Accuracy: {accuracy_val * 100:.2f}%')

# Save the model
import joblib
joblib.dump(rf_model, 'rf_model.pkl')

 
# After training on all folds, you can evaluate the model on the entire dataset
y_pred = rf_model.predict(X_test)
print(X_test.shape)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Total Accuracy: {accuracy * 100:.2f}%')




c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


113/113 ━━━━━━━━━━━━━━━━━━━━ 11s 92ms/step


Validation Accuracy: 82.01%
Validation Accuracy: 88.58%
Validation Accuracy: 88.58%
Validation Accuracy: 86.11%
Validation Accuracy: 85.07%
Validation Accuracy: 84.38%
Validation Accuracy: 82.29%
Validation Accuracy: 87.50%
Validation Accuracy: 84.03%
Validation Accuracy: 83.33%
(721, 2)
Total Accuracy: 84.19%
